In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
#from psycopg2 import extras as ex
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from sklearn.preprocessing import *
import datetime

## prepare data

In [2]:
# connect to the postgresql
db_connection = psycopg2.connect(host='127.0.0.1',dbname="chicago_business_intelligence", user="postgres" , password="12345")
cursor = db_connection.cursor()

In [3]:
# get data
select_data = (
    """
        SELECT * FROM "neighborhood_community_zip";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_1 = pd.DataFrame(result_1)
df_1.columns = ["id","geo_id","zipcode","community","neighborhood"]
df_1 = df_1[["zipcode","community","neighborhood"]]
df_1.zipcode = df_1.zipcode.astype("int")
df_1

,zipcode,community,neighborhood
0,60601,Loop,Printer's Row
1,60602,Loop,Printer's Row
2,60603,Loop,Printer's Row
3,60604,Loop,Printer's Row
4,60605,Loop,Printer's Row
...,...,...,...
306,60652,Ashburn,Parkview
307,60652,Ashburn,Scottsdale
308,60652,Ashburn,Wrightwood
309,60656,O'Hare,Schorsch Forest View


In [4]:
# get data
select_data = (
    """
        SELECT * FROM "building_permits";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_2 = pd.DataFrame(result_1)
df_2.columns = ["id","permit_id","permit_type","total_fee","latitude","longitude","zipcode"]
df_2 = df_2[["permit_id","total_fee","zipcode"]]
df_2.zipcode = df_2.zipcode.astype("int")
df_2

,permit_id,total_fee,zipcode
0,3248507,75.0,60618
1,3248478,75.0,60623
2,3248576,550.0,60629
3,3248550,625.0,60620
4,3248490,75.0,60609
...,...,...,...
993,3246783,150.0,60629
994,3217342,675.0,60607
995,3246616,75.0,60609
996,3246714,75.0,60646


In [5]:
# get data
select_data = (
    """
        SELECT * FROM "public_health_statistics";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_3 = pd.DataFrame(result_1)
df_3.columns = ["id","community_area_name","below_poverty_level","per_capita_income","unemployment"]
df_3 = df_3[["community_area_name","below_poverty_level","unemployment"]]
df_3.columns = ["community","below_poverty_level","unemployment"]
df_3

,community,below_poverty_level,unemployment
0,Rogers Park,22.7,7.5
1,West Ridge,15.1,7.9
2,Uptown,22.7,7.7
3,Lincoln Square,9.5,6.8
4,North Center,7.1,4.5
...,...,...,...
72,Washington Heights,15.7,18.3
73,Mount Greenwood,3.1,6.9
74,Morgan Park,13.7,14.9
75,O'Hare,9.5,4.7


## process data

In [6]:
pro_df_1 = df_3.merge(df_1,how='inner', on='community')
pro_df_2 = pro_df_1.merge(df_2,how='inner', on='zipcode')
pro_df_2["poverty+unemployment"] = pro_df_2["below_poverty_level"]+pro_df_2["unemployment"]
pro_df_2 = pro_df_2[["neighborhood","total_fee","poverty+unemployment"]]
pro_df_2 = pro_df_2.reset_index(drop=True)
pro_df_3 = pro_df_2.groupby(by=["neighborhood","poverty+unemployment"],dropna=False)["total_fee"].sum().reset_index()
pro_df_3

,neighborhood,poverty+unemployment,total_fee
0,Andersonville,25.6,194329.77
1,Ashburn Estates,18.3,3452.00
2,Back of the Yards,48.0,97381.78
3,Belmont Gardens,32.0,46788.15
4,Belmont Heights,16.9,8053.08
...,...,...,...
117,West Rogers Park,23.0,21338.31
118,Wicker Park,21.7,46130.49
119,Wildwood,11.6,4875.00
120,Wrightwood,18.3,3452.00


## report

In [7]:
rp_top5 = pro_df_3.sort_values(by="poverty+unemployment" , ascending=False)[:5]
rp_top5 = rp_top5.reset_index(drop=True)
rp_top5

,neighborhood,poverty+unemployment,total_fee
0,Englewood,67.0,4250.00
1,Hamilton Park,63.5,6343.40
2,Lawndale,57.1,60513.16
3,Fifth City,56.1,9384.08
4,Jackson Park Highlands,49.2,35810.40
